## Parser

In [2]:
import csv
import json

# Function to convert CSV to JSON
def convert_csv_to_json(csv_filepath, json_filepath):
    # Create a list to hold the data
    quiz_data = []

    # Open the CSV
    with open(csv_filepath, mode='r', encoding='utf-8') as csv_file:
        # Create a CSV reader object
        csv_reader = csv.DictReader(csv_file)
        
        # Iterate over the CSV rows
        for row in csv_reader:
            # Append the row to the list
            quiz_data.append(row)
    
    # Open the JSON file and write the data
    with open(json_filepath, mode='w', encoding='utf-8') as json_file:
        # Convert the list to JSON
        json_file.write(json.dumps(quiz_data, indent=4))

# File paths (you can change these to the actual file paths)
csv_filepath = 'quantum_bank.csv'
json_filepath = 'quantum_bank.json'

# Convert CSV to JSON
convert_csv_to_json(csv_filepath, json_filepath)


## Query GPT-4 for Answer

In [4]:
import openai
import os
import json

# Define some varaibles
openai.api_key = ""
file_path = 'quantum_bank.json'
answers=[]
# Function to extract and print each field from the JSON objects
def extract_fields(data):
    extracted_data = []
    for item in data:
        extracted_item = {
            "question": item['Question'],
            "answer": item['Answer'],
            "hint1": item['Hint1'],
            "hint2": item['Hint2'],
            "hint3": item['Hint3']
        }
        extracted_data.append(extracted_item)
    return extracted_data
    
with open(file_path, 'r') as file:
    data = json.load(file)
    extracted_data = extract_fields(data)
    
for item in extracted_data:
    messages = []
    answer = {}
    messages.append({"role": "system", "content": "You are QuantumGPT, a tool that answers quantum related questions. Please answer the question and write equations in latex."})
    # Getting answer without hint
    message_nh=messages
    message_nh.append({"role": "user", "content": item['question']})
    response = openai.ChatCompletion.create(
                model="gpt-4",
                temperature=1.0,
                max_tokens=1000,
                messages=message_nh,
            )
    response_str_nh=response.choices[0].message.content
    answer["No_hint"]= response_str_nh
    # Getting answer with hint 1 (best hint)
    message_h1=messages
    message_h1.append({"role": "assistant", "content": item['hint1']})
    message_h1.append({"role": "user", "content": item['question']})
    response = openai.ChatCompletion.create(
                model="gpt-4",
                temperature=1.0,
                max_tokens=1000,
                messages=message_h1,
            )
    response_str_h1=response.choices[0].message.content
    answer["Hint1"]=response_str_h1
    # Getting answer with hint 2 (vaguely related hint)
    message_h2=messages
    message_h2.append({"role": "assistant", "content": item['hint2']})
    message_h2.append({"role": "user", "content": item['question']})
    response = openai.ChatCompletion.create(
                model="gpt-4",
                temperature=1.0,
                max_tokens=1000,
                messages=message_h2,
            )
    response_str_h2=response.choices[0].message.content
    answer["Hint2"]=response_str_h2
    # Getting answer with hint 3 (unrelevant hint)
    message_h3=messages
    message_h3.append({"role": "assistant", "content": item['hint3']})
    message_h3.append({"role": "user", "content": item['question']})
    response = openai.ChatCompletion.create(
                model="gpt-4",
                temperature=1.0,
                max_tokens=1000,
                messages=message_h3,
            )
    response_str_h3=response.choices[0].message.content
    answer["Hint3"]=response_str_h3
    answer["answer"]=item['answer']
    answers.append(answer)

In [5]:
# Dump to json
# Define the function to write the list to a JSON file
def write_list_to_json(list_to_dump, json_file_path):
    with open(json_file_path, 'w', encoding='utf-8') as file:
        json.dump(list_to_dump, file, indent=4)

# Specify the path for the output JSON file
json_file_path = 'GPT4_results.json'

# Call the function to write the list to the specified JSON file
write_list_to_json(answers, json_file_path)